In [1]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time, math, pickle
import numpy as np
import argparse, pprint
import pyrealsense2 as rs

from sklearn import linear_model, datasets
from matplotlib import pyplot as plt

from scripts.hyutils.sys_utils import *
from scripts.hyutils.img_utils import *
from scripts.hyutils.debug_utils import *
from scripts.hyutils.plotting_utils import *
from scripts.hyutils.uv_mapping_utils import *
from scripts.devices.d415_camera import *
from scripts.VBOATS import VBOATS

import ipywidgets as widgets
from IPython.display import display, HTML, Math
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

In [5]:
cam.__del__()

[INFO] Closing CameraD415 object


In [2]:
cam = CameraD415(flag_save=False, use_statistics=False, fps=30,verbose=True)

('Found device: ', u'Intel RealSense D435I', ' ', u'850312072393')
[INFO] CameraD415() -- Resetting Hardware...
[INFO] CameraD415() -- Initialization successful!!!


In [3]:
intr = cam.get_intrinsics()
extr = cam.get_extrinsics()
print(intr["depth"])

focal = intr["depth"].fx
baseline = extr.translation[0]
print(focal, baseline)

vboat = VBOATS()
vboat.debug = False
vboat.dead_x = 0
vboat.dead_y = 3

width: 640, height: 480, ppx: 322.513, ppy: 245.431, fx: 386.112, fy: 386.112, model: Brown Conrady, coeffs: [0, 0, 0, 0, 0]
(386.11224365234375, 0.014579247683286667)


In [4]:
rgb,depth = cam.read()
pplot(cv2.cvtColor(rgb,cv2.COLOR_RGB2BGR),"hey")
pplot(depth,"depth")

In [ ]:
def extract_houghline(rho, theta, r_offset = 0, verbose=False):
    angle = np.rad2deg(theta)
    print("Theta = %.3f | Radius = %.3f" % (angle, rho))
    r = rho + r_offset
    a = np.cos(theta);            b = np.sin(theta) 
    x0 = a*r;                     y0 = b*r;
    x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a)) 
    x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
    return (x1,y1), (x2,y2)

def get_ground_mask(_img, show_display = True, use_probabilistic = False, verbose = False):
    disp = np.copy(_img)
    disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
    display = np.copy(disp)
    
    cpy = np.copy(_img)
    thresh = int(np.max(cpy)*0.075)
    print(thresh)
    _,tmp = cv2.threshold(cpy, 32,255,cv2.THRESH_TOZERO)
    
    dt = 0
    t0 = time.time()
#     edges = cv2.Canny(tmp,50,150,apertureSize = 3)
    edges = np.copy(tmp)
    
    lines = cv2.HoughLines(edges,15,np.pi/180, 550)
    t1 = time.time()
    dt = t1 - t0
    # print("[INFO] HoughLines() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))

    # t0 = time.time()
    # edges = cv2.Canny(tmp,1,150,apertureSize = 3)
    # linesP = cv2.HoughLinesP(edges,10,np.pi/180, 100,100,10)
    # t1 = time.time()
    # dt = t1 - t0
    # print("[INFO] HoughLinesProbabilistic() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))

    pplot(edges,"edges")
#     print(len(lines))
    
    # for x1,y1,x2,y2 in linesP[0]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    # for x1,y1,x2,y2 in linesP[1]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    # for x1,y1,x2,y2 in linesP[2]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    ############################################
    if(lines is None):
        print("No Lines found.")
        return False, _img, [], [], []
    numLines = len(lines)
    nLines = min(12, numLines)

    pts1 = []
    pts2 = []
    angles = []
    rhos = []
    for i in range(0,nLines):
        for r,theta in lines[i]:
            angle = np.rad2deg(theta)
            print("[%d] Theta = %.3f | Radius = %.3f" % (i, angle, r))
            if(show_display): 
                a = np.cos(theta);            b = np.sin(theta) 
                x0 = a*r;                     y0 = b*r;
                x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a)) 
                x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
                pt1 = (x1,y1);      pt2 = (x2,y2)
                cv2.line(display,pt1, pt2, (0,255,0),1)
            if(178.0 < angle <= 190.0): continue
            elif(-10.0 <= angle <= 10.0): continue
            elif(260.0 <= angle <= 280.0): continue
            elif(-100.0 <= angle <= -80.0): continue
            elif(-190.0 <= angle <= -170.0): continue
            else:
                if(verbose): print("[%d] Theta = %.3f | Radius = %.3f" % (i, angle, r))
                angles.append(angle)
                rhos.append(r)
                a = np.cos(theta);            b = np.sin(theta) 
                x0 = a*r;                     y0 = b*r;
                x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a)) 
                x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
                pt1 = (x1,y1);      pt2 = (x2,y2)
                pts1.append([x1,y1]);  pts2.append([x2,y2])
                if(verbose): print("[%d] Point1 = (%d, %d) | Point2 = (%d, %d)" % (i, x1,y1, x2,y2))
                if(show_display): cv2.line(display,pt1, pt2, (0,255,0),1)
    if(len(pts1) <= 0):
        print("[INFO] No Ground Present!")
        return False, cv2.cvtColor(disp,cv2.COLOR_BGR2GRAY), display, angles,rhos
    pts1 = np.array(pts1)
    pts2 = np.array(pts2)
    print(angles)
    idxMin = np.argmin(angles)
    idxMax = np.argmax(pts2[:,1])

    rgain = (rhos[idxMin]/10.0)
    again = (angles[idxMin]/180.0)
    print(rgain, again)
    tpt1, tpt2 = extract_houghline(rhos[idxMin],np.deg2rad(angles[idxMin]-again*0.75),-0.25*rgain)
#     tpt1, tpt2 = extract_houghline(rhos[idxMin],np.deg2rad(np.mean(angles)),-2)
    
    pts = np.array([
        (0, disp.shape[0]),
        (tpt1[0],tpt1[1]),
        (tpt2[0],tpt2[1]),
        (disp.shape[1], disp.shape[0]),
        (0, disp.shape[0])
    ])
    cv2.fillPoly(disp, [pts], (0,0,0))

    newVmap = cv2.cvtColor(disp,cv2.COLOR_BGR2GRAY)
    return True, newVmap, display, angles,rhos

def find_rough_ground_breakpoint(_img, verbose = False, debug = False,show_plots=False):
    h, _ = _img.shape[:2]
    hist = vertical_hist(_img)
    smHist = histogram_sliding_filter(hist,window_size=8)
    if(show_plots):
        plt.figure(1)
        plt.clf()
        plt.title('Histogram of the image')
        plt.plot(range(smHist.shape[0]), smHist[:])
    cnt = 0
    idxs = []
    lastIdx = None
    flag_started = False
    argIdx = smHist.shape[0]/4
    if(verbose): print("Starting Search Index: %d" % (argIdx))
    for i, val in enumerate(smHist[argIdx:]):
        idx = i+argIdx
        dval = val - smHist[idx-1]
        if(dval <= -15.0):
            if(lastIdx is None):
                lastIdx = idx
                idxs.append(idx)
            cnt += 1            
            flag_started = True
            if(verbose): print("[#%d] Found Negative Slope = %.1f" % (idx, dval))
        if(dval >= 0.0):
            if(flag_started):
                cnt = 0
                lastIdx = None
                if(debug): print("reseting")
                flag_started = False
        if(debug): print(idx,val, dval)
    if((cnt >= 1) and (lastIdx is not None)):
        if(debug): print("Found %d Indices with sufficient negative slopes: %s" % (len(idxs),str(idxs)))
        if(h-20 <= lastIdx <= h):
            try:
                lastIdx = idxs[-2]
                if(h-20 <= lastIdx <= h): lastIdx = -1
            except:
                lastIdx = -1
        if(verbose): print("cnt, lowest idx", cnt, lastIdx)
        return lastIdx
    else: return -1


def get_linear_ground_mask(img, debug_timings=True,show_plots=False,verbose=True):
        # img = np.copy(_img)
        h, _ = img.shape[:2]
        if(debug_timings):
            dt = 0
            t0 = time.time()
        
        lowIdx = find_rough_ground_breakpoint(img)
        if(lowIdx == -1): hm = h/2
        else: hm = lowIdx
        
        if(hm >= h-30): hm = h/2
    
        if(verbose): print("Thresholding Below %d" % (hm))
        topHalf = img[0:hm, :]
        botHalf = img[hm:h, :]
        _,ttHalf = cv2.threshold(topHalf, 225,255,cv2.THRESH_TOZERO)
        _,tbHalf = cv2.threshold(botHalf, 64,255,cv2.THRESH_TOZERO)
        tmp = np.concatenate((ttHalf,tbHalf), axis=0)

        nonzero = tmp.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        nonzerox = np.reshape(nonzerox,(nonzerox.shape[0],1))

        line_X = np.arange(nonzerox.min(), nonzerox.max())[:, np.newaxis]
        
#         lr = linear_model.LinearRegression()
#         lr.fit(nonzerox,nonzeroy)
#         line_y = lr.predict(line_X)

        ransac = linear_model.RANSACRegressor(stop_probability=0.85,max_trials=30)
        ransac.fit(nonzerox, nonzeroy)
        inlier_mask = ransac.inlier_mask_
        outlier_mask = np.logical_not(inlier_mask)

        line_y_ransac = ransac.predict(line_X)
        
        if(debug_timings):
            t1 = time.time()
            dt = t1 - t0
            print("[INFO] get_linear_ground_mask() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))
            
        if(show_plots):
            pplot(tmp,"Image")
            plt.clf()
            pplot(tmp,"Image")

            plt.scatter(nonzerox,nonzeroy, color='yellowgreen', marker='.',label='Inliers')
            # plt.plot(line_X, line_y, color='white', linewidth=2, label='Linear regressor')
            plt.plot(line_X, line_y_ransac, color='red', linewidth=2, label='RANSAC regressor')
            
        m = ransac.estimator_.coef_
        b = ransac.estimator_.intercept_
        print("Estimated coefficients (RANSAC): m = %.3f | b = %.3f" % (m,b))
        return m,b, line_X,line_y_ransac

In [ ]:
rgb,depth = cam.read()
test_depth = np.copy(depth)

t0 = time.time()
norm = cv2.normalize(depth, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
loc = np.where(norm == 0.0)
norm[loc] = 1.0
disparity = (focal*baseline)/norm
disparity[loc] = 0.0
disparity = np.uint8(disparity)
dmax0 = np.max(disparity); dmin0 = np.min(disparity)
t1 = time.time()
dt = t1 - t0
print("[%.2f] Normalized Disparity Limits: %.2f, %.2f" % (1/dt,dmax0,dmin0))

t0 = time.time()
test_depth = test_depth*0.001
loc2 = np.where(test_depth == 0.0)
test_depth[loc2] = 1.0
testD = (focal*baseline)/test_depth
testD[loc2] = 0.0
disparity2uintGain = 255/np.max(testD)
testD = np.uint8(testD*disparity2uintGain)
dmax1 = np.max(testD); dmin1 = np.min(testD)
t1 = time.time()
dt = t1 - t0
print("[%.2f] Test Disparity Limits: %.2f, %.2f" % (1/dt, dmax1,dmin1))

pplot(depth,"raw depth")
pplot(norm,"normalized depth")
pplot(disparity,"disparity from norm")

pplot(testD,"test disparity")

In [ ]:
rgb,depth = cam.read()
pplot(cv2.cvtColor(rgb,cv2.COLOR_RGB2BGR),"rgb")

tmp = depth*0.001
loc = np.where(tmp == 0.0)
tmp[loc] = 1.0
dipsarity = (focal*baseline)/tmp
dipsarity[loc] = 0.0
disparity2uintGain = 255/np.max(dipsarity)
dipsarity = np.uint8(dipsarity*disparity2uintGain)

# kernelI = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
# disparity = cv2.morphologyEx(dipsarity, cv2.MORPH_CLOSE, kernelI)
disparity = dipsarity
pplot(disparity,"disparity")
raw_umap, raw_vmap, dt = vboat.get_uv_map(disparity)
pplot(raw_umap,"Raw Umap")
pplot(raw_vmap,"Raw Vmap")

umap = np.copy(raw_umap)
vmap = np.copy(raw_vmap)

###########################################
cv2.rectangle(vmap,(0,0),(1, vmap.shape[0]),(0,0,0), cv2.FILLED)
# thresh0 = int(np.max(vmap)*0.025)
# _,tmp = cv2.threshold(vmap, thresh0,255,cv2.THRESH_TOZERO)

pplot(vmap,"Vmap")
display = np.copy(vmap)
display = cv2.cvtColor(display,cv2.COLOR_GRAY2BGR)
###########################################
m,b, xs,ys = get_linear_ground_mask(vmap,show_plots=True)
if(m >= 0.0):
    h,w = vmap.shape[:2]
    yf = int(m*w + b)
    pt1 = (0,int(b))
    pt2 = (w,yf-15)
    pts = np.array([(0, h),pt1,pt2,(w, h),(0, h)])
    cv2.fillPoly(display, [pts], (0,0,0))
    # cv2.line(display,pt1, pt2, (0,255,255),1)
    pplot(display,"ground line")
else: print("No Ground Detected.")
#######################################
# isGroundPresent, new_vmap, line_display, angs, rs = get_ground_mask(tmp)
# pplot(new_vmap,"new vmap")


In [ ]:
over = make_uv_overlay(dipsarity,raw_umap,raw_vmap)
pplot(over,'Overlay')

In [ ]:
# ------------- GROUND SEGMENTATION ------------------
def groundSeg(filtered_vmap, stopP = 0.85, maxTrials=30):
    gndImg = np.copy(filtered_vmap)
    nonzero = gndImg.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    nonzerox = np.reshape(nonzerox,(nonzerox.shape[0],1))

    ransac = linear_model.RANSACRegressor(stop_probability=stopP,max_trials=maxTrials)
    ransac.fit(nonzerox, nonzeroy)
    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)

    line_X = np.arange(nonzerox.min(), nonzerox.max())[:, np.newaxis]
    line_y_ransac = ransac.predict(line_X)

    if(1):
        plt.figure("RANSAC")
        plt.clf()
        plt.subplots_adjust(wspace=0.0,hspace=0.0,left=0.0,right=1.0,top=1.0, bottom=0.0)
        plt.axis([0.0, 255.0, 480, 0])
        plt.imshow(gndImg,interpolation='bilinear')
        plt.scatter(nonzerox,nonzeroy, color='yellowgreen', marker='.',label='Inliers')
        plt.plot(line_X, line_y_ransac, color='red', linewidth=2, label='RANSAC regressor')
        plt.show()

    m = ransac.estimator_.coef_
    b = ransac.estimator_.intercept_
    print("Estimated coefficients (RANSAC): m = %.3f | b = %.3f" % (m,b))

In [ ]:
# ------------- GET NEW DEPTH IMAGE ------------------
rgb,depth = cam.read()
pplot(rgb,"rgb")
tmp = depth*0.001
loc = np.where(tmp == 0.0)
tmp[loc] = 1.0
dipsarity = (focal*baseline)/tmp
dipsarity[loc] = 0.0
disparity2uintGain = 255/np.max(dipsarity)
dipsarity = np.uint8(dipsarity*disparity2uintGain)
disparity = dipsarity

# kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
# disparity = cv2.morphologyEx(disparity, cv2.MORPH_CLOSE, kernel)

pplot(disparity,"disparity")
raw_umap, raw_vmap, dt = vboat.get_uv_map(disparity)
umap = np.copy(raw_umap)
vmap = np.copy(raw_vmap)

pplot(vmap,"Raw Vmap")
cv2.rectangle(vmap,(0,0),(1, vmap.shape[0]),(0,0,0), cv2.FILLED)

# ------------- THRESHOLD V STRIPS ------------------
vMax = np.max(vmap)
nThreshs = int(math.ceil((vMax/256.0)) * 4)
print(vMax,nThreshs)

stripsV = strip_image(vmap, nstrips=nThreshs, horizontal_strips=False)
pplots(stripsV,"vmap strips",(1,nThreshs))

# thresholds = [0.95,0.35,0.35,0.5]
thresholds = [0.85,0.85,0.75,0.5]

stripsPV = []
for i, strip in enumerate(stripsV):
    tmpMax = np.max(strip)
    tmpMean = np.mean(strip)
    tmpStd = np.std(strip)
    if(tmpMean == 0):
        stripsPV.append(strip)
        continue
    tmp = (vMax - tmpMean)/tmpStd
    tmp2 = (vMax- tmpStd)/ tmpMean
    print("---------- [Strip %d] ---------- \r\n\tMax = %.1f, Mean = %.1f, Std = %.1f" % (i,tmpMax,tmpMean,tmpStd))
    dratio = vMax/255.0
    relRatio = (tmpMax-tmpStd)/float(vMax)
    rrelRatio = (tmpMean)/float(tmpMax)
    print("\tRatios: %.3f, %.3f, %.3f" % (dratio, relRatio, rrelRatio))
#     if(dratio >= 0.4): gain = (rrelRatio*thresholds[i] + relRatio*dratio)
#     else: gain = 1.0 - (rrelRatio + relRatio*dratio)*thresholds[i]
    # gain = (1.0 - (thresholds[i]*relRatio))*dratio
    if(relRatio >= 0.4): gain = relRatio + rrelRatio
    else: gain = 1.0 - (relRatio + rrelRatio)
    thresh = int(thresholds[i]* gain * tmpMax)
    print("\tGain = %.2f, Thresh = %d" % (gain,thresh))
    _, tmpStrip = cv2.threshold(strip, thresh,255,cv2.THRESH_TOZERO)
#     _, tmpStrip = cv2.threshold(strip, thresh,255,cv2.THRESH_BINARY)
    stripsPV.append(tmpStrip)

pplots(stripsPV,"thresholded vmap strips",(1,nThreshs))

filtV = np.concatenate(stripsPV, axis=1)
pplot(filtV,"Filtered Vmap")
# ------------- GROUND SEGMENTATION ------------------
groundSeg(filtV)

In [ ]:
groundSeg(filtV, stopP= 0.75,maxTrials=100)

In [ ]:
idx = np.argmin(angs)
dum = cv2.cvtColor(tmp,cv2.COLOR_GRAY2BGR)
rgain = (rs[idx]/10.0)
again = (angs[idx]/180.0)
print(again)
print(rgain)

pt1, pt2 = extract_houghline(rs[idx],np.deg2rad(angs[idx]))
tpt1, tpt2 = extract_houghline(rs[idx],np.deg2rad(angs[idx]-again*0.75),-0.25*rgain)
cv2.line(dum,pt1, pt2, (0,255,0),1)
cv2.line(dum,tpt1, tpt2, (0,0,255),1)
pplot(dum,"test lines")

In [ ]:
style = {'description_width': 'initial'}
layStretch = Layout(flex='1 1 auto')
layCenter = Layout(align_self='center',display = 'flex',align_items='stretch',flex='1 1 auto')
layWidth = Layout(width="auto",align_self='center')

lblSz1 = widgets.Label(value="Hough Thresholds", layout=layCenter)
sz1Selector = widgets.IntSlider(value=10,min=0,max=100,step=1,description=' ', layout = layCenter)
sz2Selector = widgets.IntSlider(value=550,min=0,max=1000,step=1,description=' ', layout = layCenter)

lblSz2 = widgets.Label(value="Probablistic Hough Thresholds:", layout=layCenter)
sz3Selector = widgets.IntSlider(value=100,min=0,max=1000,step=1,description='minLineLength', layout = layStretch)
sz4Selector = widgets.IntSlider(value=10,min=0,max=1000,step=1,description='maxLineGap', layout = layStretch)

lblU = widgets.Label(value="UMap Strip Thresholds (0,1):", layout=layCenter)
u1threshSelector = widgets.IntSlider(value=180,min=1,max=360,step=1,description='Angle',layout= layStretch)
# u1threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ',layout= layStretch)
u2threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout = layStretch)

lblV = widgets.Label(value="VMap Strip Thresholds (0,1):", layout=layCenter)
v1threshSelector = widgets.IntSlider(value=5,min=0,max=255,step=1,description=' ',layout = layStretch)
v2threshSelector = widgets.IntSlider(value=70,min=0,max=255,step=1,description=' ',layout = layStretch)

lblD = widgets.Label(value="Deadzone Thresholds (U1, U2, V):", layout=layCenter)
d1threshSelector = widgets.FloatSlider(value=0.25,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d2threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d3threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)

lblK = widgets.Label(value="Kernel Sizes:", layout=layCenter)
k11Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k11', layout = layStretch)
k12Selector = widgets.IntSlider(value=2,min=0,max=100,step=1,description='k12', layout = layStretch)
k21Selector = widgets.IntSlider(value=50,min=0,max=100,step=1,description='k21', layout = layStretch)
k22Selector = widgets.IntSlider(value=5,min=0,max=100,step=1,description='k22', layout = layStretch)
k31Selector = widgets.IntSlider(value=45,min=0,max=100,step=1,description='k31', layout = layStretch)
k32Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k32', layout = layStretch)

win1Selector = widgets.IntSlider(value=10,min=0,max=100,step=1,description='win1', layout = layStretch)
win1ThreshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)


flag1 = widgets.Checkbox(value=False, description='Use Test Pipeline', layout = layStretch)
flag2 = widgets.Checkbox(value=True, description='Show Computational Timing', layout = layStretch)
flagt2 = widgets.Checkbox(value=False, description='Show Misc Timings', layout = layStretch)
flag3 = widgets.Checkbox(value=True, description='Use Filtered Overlay', layout = layStretch)
flag4 = widgets.Checkbox(value=False, description='Show Colorful Obstacle Overlay', layout = layStretch)

In [ ]:
def thresholding(dx=0,dy=3, sz1=5,sz2=10,tu0=1,tu1=0.1,tv0=5, tv1=70, tdu = 0.4, tdu2 = 0.1, tdv = 0.15,
    k11=15,k12=2,k21=50,k22=5,k31=75,k32=1, win1=5, win1thr = 0.1, searchWin=10, flag_test=True,
    flag_timing=False,flag_timing2=False,flag3=True,flag4=True, avg_flag=False):
    
    _img = np.copy(vmap)
    disp = np.copy(_img)
    disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
    display = np.copy(disp)
    
    cpy = np.copy(_img)
    thresh = int(np.max(cpy)*0.075)
    print(thresh)
    _,tmp = cv2.threshold(cpy, 64,255,cv2.THRESH_TOZERO)
    
    dt = 0
    t0 = time.time()
    
#     edges = cv2.Canny(tmp,50,150,apertureSize = 3)
    edges = cpy
    if(flag_test): lines = cv2.HoughLinesP(edges,dx,np.pi/tu0, dy,sz1,sz2) 
    else: lines = cv2.HoughLines(edges,dx,np.pi/tu0, dy)
    # edges = cv2.Canny(tmp,50,150,apertureSize = 3)
    
    
#     t1 = time.time()
#     dt = t1 - t0
#     print("[INFO] HoughLines() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))
#     print("[INFO] HoughLinesProbabilistic() --- Took %f seconds (%.2f Hz) to complete" % (dt, 1/dt))

#     pplot(edges,"edges")
#     print(len(lines))
    
    # for x1,y1,x2,y2 in linesP[0]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    # for x1,y1,x2,y2 in linesP[1]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    # for x1,y1,x2,y2 in linesP[2]:
    #     cv2.line(disp,(x1,y1),(x2,y2),(0,255,0),2)
    ############################################
    if(lines is None): print("No Lines found.")
    numLines = len(lines)
    nLines = min(6, numLines)

    pts1 = []
    pts2 = []
    angles = []
    rhos = []
    if(flag_test):
        for i in range(0,nLines):
            for x1,y1,x2,y2 in lines[i]:
                cv2.line(display,(x1,y1),(x2,y2),(0,255,0),2)
    else:
        for i in range(0,nLines):
            for r,theta in lines[i]:
                angle = np.rad2deg(theta)
                print("[%d] Theta = %.3f | Radius = %.3f" % (i, angle, r))
    #             a = np.cos(theta);            b = np.sin(theta) 
    #             x0 = a*r;                     y0 = b*r;
    #             x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a)) 
    #             x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
    #             pt1 = (x1,y1);      pt2 = (x2,y2)
    #             cv2.line(display,pt1, pt2, (0,255,0),1)
                if(178.0 <= angle <= 190.0): continue
                elif(-10.0 <= angle <= 10.0): continue
                elif(260.0 <= angle <= 280.0): continue
                elif(-100.0 <= angle <= -80.0): continue
                elif(-190.0 <= angle <= -170.0): continue
                elif(0.0 <= angle <= 60.0): continue
                else:
                    print("[%d] Theta = %.3f | Radius = %.3f" % (i, angle, r))
                    angles.append(angle)
                    rhos.append(r)
                    a = np.cos(theta);            b = np.sin(theta) 
                    x0 = a*r;                     y0 = b*r;
                    x1 = int(x0 + 1000*(-b));     y1 = int(y0 + 1000*(a)) 
                    x2 = int(x0 - 1000*(-b));     y2 = int(y0 - 1000*(a))
                    pt1 = (x1,y1);      pt2 = (x2,y2)
                    pts1.append([x1,y1]);  pts2.append([x2,y2])
                    print("[%d] Point1 = (%d, %d) | Point2 = (%d, %d)" % (i, x1,y1, x2,y2))
                    cv2.line(display,pt1, pt2, (0,255,0),1)
        pts1 = np.array(pts1)
        pts2 = np.array(pts2)
        print(angles)
        idxMin = np.argmin(angles)
        idxMax = np.argmax(pts2[:,1])

        rgain = (rhos[idxMin]/10.0)
        again = (angles[idxMin]/180.0)
        print(rgain, again)
        tpt1, tpt2 = extract_houghline(rhos[idxMin],np.deg2rad(angles[idxMin]-again*0.75),-0.25*rgain)
        cv2.line(display,tpt1, tpt2, (0,0,255),1)
    pplot(display,"display")

In [ ]:
p = interactive(thresholding,
    dx = sz1Selector, dy=sz2Selector, sz1=sz3Selector,sz2=sz4Selector,
    tu0 = u1threshSelector,tu1 = u2threshSelector, tv0 = v1threshSelector, tv1 = v2threshSelector,
    tdu = d1threshSelector, tdu2=d2threshSelector, tdv = d3threshSelector, k11=k11Selector,k12=k12Selector,
    k21=k21Selector,k22=k22Selector,k31=k31Selector,k32=k32Selector,win1=win1Selector,win1thr=win1ThreshSelector,
    searchWin=(0,40,1), flag_test = flag1, flag_timing = flag2, flag_timing2 = flagt2, flag3 = flag3, flag4 = flag4)

rows = []
rows.append(HBox([p.children[0]]))
rows.append(HBox([lblSz1,p.children[1], p.children[2]], layout = layStretch))
rows.append(HBox([lblSz2,p.children[3], p.children[4]], layout = layStretch))
rows.append(HBox([lblU, p.children[5], p.children[6]], layout = layStretch))
rows.append(HBox([lblV, p.children[7], p.children[8]], layout = layStretch))
rows.append(HBox([lblD, p.children[9], p.children[10], p.children[11]], layout = layStretch))
rows.append(HBox([p.children[12], p.children[13]], layout = layStretch))
rows.append(HBox([p.children[14], p.children[15]], layout = layStretch))
rows.append(HBox([p.children[16], p.children[17]], layout = layStretch))
rows.append(HBox([p.children[18], p.children[19]], layout = layStretch))

rows.append(HBox([p.children[-7],p.children[-6],p.children[-5], p.children[-4]], layout = layStretch))
rows.append(HBox([p.children[-3], p.children[-2]], layout = layStretch))

controls = VBox(rows, layout = Layout(display='flex'))
output = p.children[-1]

tabs = widgets.Tab([controls, output])
tabs.set_title(0, 'Parameters')
tabs.set_title(1, 'StdOuts')
# display(tabs)

display(VBox([controls, output]))